In [1]:
sc

<SparkContext master=local[2] appName=PySparkShell>

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions


In [4]:
def loadMovieNames():
    movieNames = {}
    with open("ml-100k/u.item",encoding ="ISO-8859-1") as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
    return movieNames

# Load up our movie ID -> name dictionary
nameDict = loadMovieNames()


In [9]:
lines = sc.textFile("ml-100k/u.data")

In [10]:
# Convert it to a RDD of Row objects
movies = lines.map(lambda x: Row(movieID =int(x.split()[1])))

In [11]:
movies.take(5)

[Row(movieID=242),
 Row(movieID=302),
 Row(movieID=377),
 Row(movieID=51),
 Row(movieID=346)]

In [12]:
# Convert that to a DataFrame
movieDataset = spark.createDataFrame(movies)

In [20]:
movieDataset.groupBy("movieID").count().show()

+-------+-----+
|movieID|count|
+-------+-----+
|    474|  194|
|     29|  114|
|     26|   73|
|    964|    9|
|   1677|    1|
|     65|  115|
|    191|  276|
|   1224|   12|
|    558|   70|
|   1010|   44|
|    418|  129|
|   1277|   19|
|   1258|   23|
|    541|   49|
|   1360|    2|
|    222|  365|
|    938|   25|
|    293|  147|
|    270|  136|
|   1127|   11|
+-------+-----+
only showing top 20 rows



In [23]:
movieDataset.groupBy("movieID").count().orderBy("count", ascending=False).show()

+-------+-----+
|movieID|count|
+-------+-----+
|     50|  583|
|    258|  509|
|    100|  508|
|    181|  507|
|    294|  485|
|    286|  481|
|    288|  478|
|      1|  452|
|    300|  431|
|    121|  429|
|    174|  420|
|    127|  413|
|     56|  394|
|      7|  392|
|     98|  390|
|    237|  384|
|    117|  378|
|    172|  367|
|    222|  365|
|    313|  350|
+-------+-----+
only showing top 20 rows



In [24]:
# Some SQL-style magic to sort all movies by popularity in one line!
topMovieIDs = movieDataset.groupBy("movieID").count().orderBy("count", ascending=False).cache()

In [25]:

# Grab the top 10
top10 = topMovieIDs.take(10)


In [26]:
# Print the results
print("\n")
for result in top10:
    # Each row has movieID, count as above.
    print("%s: %d" % (nameDict[result[0]], result[1]))




Star Wars (1977): 583
Contact (1997): 509
Fargo (1996): 508
Return of the Jedi (1983): 507
Liar Liar (1997): 485
English Patient, The (1996): 481
Scream (1996): 478
Toy Story (1995): 452
Air Force One (1997): 431
Independence Day (ID4) (1996): 429
